In [31]:
# 立ち上げ

import sys
sys.path.append("pybcapclient")

import pybcapclient.bcapclient as bcapclient
import time
import math


host = "192.168.56.11"
port = 5007
timeout = 2000

Name = ""
Provider="CaoProv.DENSO.VRC"
Machine = ("localhost")
Option = ("")


### Connection processing of tcp communication
m_bcapclient = bcapclient.BCAPClient(host,port,timeout)

### start b_cap Service
m_bcapclient.service_start("")

### Connect to RC8 (RC8(VRC)provider)
hCtrl = m_bcapclient.controller_connect(Name,Provider,Machine,Option)
### get Robot Object Handl
HRobot = m_bcapclient.controller_getrobot(hCtrl,"Arm","")

m_bcapclient.robot_execute(HRobot, "ManualResetPreparation", "")
m_bcapclient.robot_execute(HRobot, "Motionpreparation", "")

### TakeArm
Command = "TakeArm"
Param = [0,0]
m_bcapclient.robot_execute(HRobot,Command,Param)

###Motor On
Command = "Motor"
Param = [1,0]
m_bcapclient.robot_execute(HRobot,Command,Param)

###set ExtSpeed Speed,Accel,Decel
Command = "ExtSpeed"
Speed = 100
Accel = 100
Decel = 100
Param = [Speed,Accel,Decel]
m_bcapclient.robot_execute(HRobot,Command,Param)

# ##set IO25
# IOHandl = m_bcapclient.controller_getvariable(hCtrl, "IO25", "")
# # write value
# m_bcapclient.variable_putvalue(IOHandl, True)
# # Disconnect
# if(IOHandl != 0):
#     m_bcapclient.variable_release(IOHandl)
# ##set IO26
# IOHandl = m_bcapclient.controller_getvariable(hCtrl, "IO26", "")
# # write value
# m_bcapclient.variable_putvalue(IOHandl, True)
# # Disconnect
# if(IOHandl != 0):
#     m_bcapclient.variable_release(IOHandl)


In [4]:
# コマンド定義

# variable
COMP = 1
COBOTTA_SLEEP_POSITION = [290,-44.5,17,-90,0,-90,5]
FLOOR_HEIGHT = 20
TOWER_DELTA_Y = 50


# general
def disconnect():
    Command = "Motor"
    Param = [0,0]
    m_bcapclient.robot_execute(HRobot,Command,Param)

    ###Give Arm
    Command = "GiveArm"
    Param = None
    m_bcapclient.robot_execute(HRobot,Command,Param)
    #Disconnect
    if(HRobot != 0):
        m_bcapclient.robot_release(HRobot)
    #End If
    if(hCtrl != 0):
        m_bcapclient.controller_disconnect(hCtrl)
    #End If
    m_bcapclient.service_stop()
    
    print('Disconnection succeeded')

def auto_calibration():
    try:
        disconnect()
    except:
        pass
    # Connection processing of tcp communication
    m_bcapclient = bcapclient.BCAPClient(host, port, timeout)
    print("Open Connection")

    # start b_cap Service
    m_bcapclient.service_start("")
    print("Send SERVICE_START packet")

    # set Parameter
    Name = ""
    Provider = "CaoProv.DENSO.VRC"
    Machine = "localhost"
    Option = ""

    try:
        # Connect to RC8 (RC8(VRC)provider) , Get Controller Handle
        hCtrl = m_bcapclient.controller_connect(Name, Provider, Machine, Option)
        print("Connect RC8")
        # Get Robot Handle
        HRobot = m_bcapclient.controller_getrobot(hCtrl, "Arm", "")

        m_bcapclient.robot_execute(HRobot, "AutoCal", "")
        m_bcapclient.robot_execute(HRobot, "MotionPreparation", "")
        m_bcapclient.robot_execute(HRobot, "TakeArm")
        m_bcapclient.robot_execute(HRobot, "ExtSpeed", 100)
        m_bcapclient.robot_move(HRobot, 1, "@P J(0,0,90,0,90,0)")
        time.sleep(1)
        go_to_joint_parameter([15.7,46.3,99.6,-22,-60,-73.7])
        go_to_position_parameter(COBOTTA_SLEEP_POSITION)



    except Exception as e:
        print('=== ERROR Description ===')
        if str(type(e)) == "<class 'pybcapclient.orinexception.ORiNException'>":
            print(e)
            errorcode_int = int(str(e))
            if errorcode_int < 0:
                errorcode_hex = format(errorcode_int & 0xffffffff, 'x')
            else:
                errorcode_hex = hex(errorcode_int)
            print("Error Code : 0x" + str(errorcode_hex))
            error_description = m_bcapclient.controller_execute(
                hCtrl, "GetErrorDescription", errorcode_int)
            print("Error Description : " + error_description)
        else:
            print(e)

    finally:
        # DisConnect
        if(HRobot != 0):
            m_bcapclient.robot_release(HRobot)
            print("Release Robot Handle")
        # End If
        if(hCtrl != 0):
            m_bcapclient.controller_disconnect(hCtrl)
            print("Release Controller")
        # End If
        m_bcapclient.service_stop()
        print("B-CAP service Stop")
        print("Auto Calibration Completed")

def change_speed(speed=100,accel=100,decel=100):
    Command = "ExtSpeed"
    Param = [speed,accel,decel]
    m_bcapclient.robot_execute(HRobot,Command,Param)
    print("Speed Parameters ","speed:",speed,"accel:",accel,"decel:",decel)

def get_now_position():
    return m_bcapclient.robot_execute(HRobot, "CurPos")

def get_now_joint():
    return m_bcapclient.robot_execute(HRobot, "CurJnt")

def clear_error():
    m_bcapclient.controller_execute(hCtrl,"ManualResetPreparation","")
    m_bcapclient.controller_execute(hCtrl,"ClearError","")

def motion_preparation():
    m_bcapclient.robot_execute(HRobot, "ManualResetPreparation", "")
    m_bcapclient.robot_execute(HRobot, "Motionpreparation", "")

def get_error_information():
    return m_bcapclient.controller_execute(hCtrl,"GetCurErrorInfo",0)[0]

def get_eroor_count():
    return m_bcapclient.controller_execute(hCtrl,"GetCurErrorCount",0)

def get_cobotta_status():
    pos=get_now_position()
    if 280<pos[0]<300 and -50<pos[1]<-40 and 10<pos[2]<20 and -100<pos[3]<-80 and -10<pos[4]<10 and -100<pos[5]<-80:
        return "Sleep"
    else:
        return "Active"

# IO
def read_IO_value(IO_No):
    IOHandl = 0
    IOHandl = m_bcapclient.controller_getvariable(hCtrl, f"IO{IO_No}", "")
    # read value
    retIO = m_bcapclient.variable_getvalue(IOHandl)
    # Disconnect
    if(IOHandl != 0):
        m_bcapclient.variable_release(IOHandl)
    m_bcapclient.service_stop()
    return retIO

def write_IO_value(IO_No,IO_Value):
    IOHandl = 0
    IOHandl = m_bcapclient.controller_getvariable(hCtrl, f"IO{IO_No}", "")

    # write value
    m_bcapclient.variable_putvalue(IOHandl, IO_Value)
    # read value
    retIO = m_bcapclient.variable_getvalue(IOHandl)
    # Disconnect
    if(IOHandl != 0):
        m_bcapclient.variable_release(IOHandl)
    m_bcapclient.service_stop()
    return retIO


# hand
def open_hand():
    param=[29.91,50]
    m_bcapclient.controller_execute(hCtrl,"HandMoveA",param)
    print("Hand Open")

def close_hand(force=6):
    param=[force,True,"DetectOn"]
    m_bcapclient.controller_execute(hCtrl,"HandMoveH",param)
    print("Hand Close")

def adjust_hand():
    now_pos= m_bcapclient.robot_execute(HRobot, "CurJnt")
    if now_pos[5]>-45:
        move_joint6(-90)
    else:
        move_joint6(90)


# move
def go_to_position_parameter(param):
    Pose = [param,"P","@E"]
    m_bcapclient.robot_move(HRobot,COMP,Pose,"")

def go_to_position_parameter_at_P(param):
    Pose = [param,"P","@P"]
    m_bcapclient.robot_move(HRobot,COMP,Pose,"")

def go_to_joint_parameter(param):
    Pose = [param,"J","@E"]
    m_bcapclient.robot_move(HRobot,COMP,Pose,"")

def go_to_position_variable(P_No):
    Pose = "P"+str(P_No)
    m_bcapclient.robot_move(HRobot,COMP,Pose,"")
    return m_bcapclient.robot_execute(HRobot, "CurPos")

def go_to_joint_variable(J_No):
    Joint = "J"+str(J_No)
    m_bcapclient.robot_move(HRobot,COMP,Joint,"")
    return m_bcapclient.robot_execute(HRobot, "CurJnt")

def shift_parallel(x=0,y=0,z=0,rx=0,ry=0,rz=0):
    now_pos= m_bcapclient.robot_execute(HRobot, "CurPos")
    target_pos=[now_pos[0]+x,now_pos[1]+y,now_pos[2]+z,now_pos[3]+rx,now_pos[4]+ry,now_pos[5]+rz]
    Pose = [target_pos,"P","@E"]
    m_bcapclient.robot_move(HRobot,COMP,Pose,"")
    now_pos= m_bcapclient.robot_execute(HRobot, "CurPos")

def move_joint1(param):
    joint_pos=[param]
    param=[joint_pos,"J","@E"]
    m_bcapclient.robot_execute(HRobot, "DriveEx",param)
    now_pos= m_bcapclient.robot_execute(HRobot, "CurJnt")

def move_joint2(param):
    joint_pos=[0,param,0,0,0,0]
    param=[joint_pos,"J","@E"]
    m_bcapclient.robot_execute(HRobot, "DriveEx",param)
    now_pos= m_bcapclient.robot_execute(HRobot, "CurJnt")

def move_joint3(param):
    joint_pos=[0,0,param,0,0,0]
    param=[joint_pos,"J","@E"]
    m_bcapclient.robot_execute(HRobot, "DriveEx",param)
    now_pos= m_bcapclient.robot_execute(HRobot, "CurJnt")

def move_joint4(param):
    joint_pos=[0,0,0,param,0,0,0]
    param=[joint_pos,"J","@E"]
    m_bcapclient.robot_execute(HRobot, "DriveEx",param)
    now_pos= m_bcapclient.robot_execute(HRobot, "CurJnt")

def move_joint5(param):
    joint_pos=[0,0,0,0,param,0]
    param=[joint_pos,"J","@E"]
    m_bcapclient.robot_execute(HRobot, "DriveEx",param)
    now_pos= m_bcapclient.robot_execute(HRobot, "CurJnt")


def move_joint6(param):
    joint_pos=[0,0,0,0,0,param]
    param=[joint_pos,"J","@E"]
    m_bcapclient.robot_execute(HRobot, "DriveEx",param)
    now_pos= m_bcapclient.robot_execute(HRobot, "CurJnt")

def go_to_sleep_position():
    go_to_position_parameter(COBOTTA_SLEEP_POSITION)

def move_in_a_spiral(center,radius_max,rotation_number):
    for i in range(0,360*rotation_number,10):
        x = center[0] + radius_max/(rotation_number*2*math.pi)*math.radians(i)*math.cos(math.radians(i))
        y = center[1] + radius_max/(rotation_number*2*math.pi)*math.radians(i)*math.sin(math.radians(i))
        go_to_position_parameter_at_P([x,y,center[2]-10,center[3],center[4],center[5],center[6]])
    shift_parallel(z=5)


# door
def open_door():
    if get_cobotta_status()=="Sleep":
        write_IO_value(25,False)
        write_IO_value(26,True)
        while read_IO_value(9)==False:
            time.sleep(0.1)
        else:
            write_IO_value(25,True)
            write_IO_value(26,True)
            print("Door Open")
        if get_error_information()==-2075524850:
            clear_error()
            motion_preparation()
    else:
        print("Door Open Error")

def close_door():
    if get_cobotta_status()=="Sleep":
        write_IO_value(25,True)
        write_IO_value(26,False)
        while read_IO_value(8)==False:
            time.sleep(0.1)
        else:
            write_IO_value(25,True)
            write_IO_value(26,True)
            print("Door Close")
        if get_error_information()==-2075524850:
            clear_error()
            motion_preparation()
    else:
        print("Door Close Error")

def get_door_status():
    door_status=(read_IO_value(8),read_IO_value(9))
    if door_status==(False,True):
        return "Open"
    elif door_status==(True,False):
        return "Close"
    elif door_status==(False,False):
        return "Neutral"
    elif door_status==(True,True):
        return "Error"


# tower floor=0~11
positions_for_miniflex=[
    [268,-49,410,-90,0,-90,261],
    [-50,-268,411,-90,0,-180,261],
    [-61.4,-327.8,382.4,-90,0,-180,261],
    [-11.25,-333.26,382.43,-90,-0.01,-170.08,261],
    [-10.75,-333.29,377.61,-89.99,-0.01,-169.98,261],
    [-10.81,-333.29,388.98,-89.99,-0.002,-169.99,261],
    ]
position_before_miniflex = [294.92,96.04,252.55,-90,0,-73,5.0]

def open_tower(floor):
    change_speed(100,100,100)
    home = [343.36,28.08,19.84+FLOOR_HEIGHT*floor,-87.43,89.41,-80.85,5.0]
    go_to_position_parameter(home)
    move_joint1(-7.5)
    shift_parallel(0,0,4,0,0,0)
    change_speed(20,100,100)
    move_joint1(14.65)
    time.sleep(0.5)
    move_joint1(-0.5)
    change_speed(100,100,100)
    shift_parallel(0,0,-5,0,0,0)
    move_joint1(5)
    shift_parallel(0,0,10,0,0,0)
    shift_parallel(-70,0,0,0,0,0)
    adjust_hand()

def close_tower(floor):
    change_speed(100,100,100)
    home = [263.04,88.39,29.40+FLOOR_HEIGHT*floor,-89.95,-0.60,-73.22,5.0]
    go_to_position_parameter(home)
    adjust_hand()
    shift_parallel(70,0,0,0,0,0)
    change_speed(20,100,100)
    move_joint1(-17)
    time.sleep(0.1)
    change_speed(100,100,100)
    move_joint1(17)
    adjust_hand()
    go_to_position_parameter(home)

def test_place_on_tower(floor,position):
    waiting_position = [310.11,5.62,32.55+FLOOR_HEIGHT*floor,-90,0,-90,5.0]
    open_tower(floor)
    move_joint1(-17)
    go_to_position_parameter(waiting_position)
    shift_parallel(x=51,y=-TOWER_DELTA_Y*position)
    change_speed(10,100,100)
    shift_parallel(z=-5)
    time.sleep(0.5)
    shift_parallel(z=5)
    change_speed(100,100,100)
    shift_parallel(x=-51,y=TOWER_DELTA_Y*position)
    go_to_position_parameter(waiting_position)
    move_joint1(17)
    close_tower(floor)

def place_on_tower(floor,position):
    waiting_position = [310.11,5.62,32.55+FLOOR_HEIGHT*floor,-90,0,-90,5.0]
    go_to_position_parameter(waiting_position)
    shift_parallel(x=51,y=-TOWER_DELTA_Y*position)
    change_speed(10,100,100)
    shift_parallel(z=-5)
    time.sleep(0.5)
    shift_parallel(z=5)
    change_speed(100,100,100)
    shift_parallel(x=-51,y=TOWER_DELTA_Y*position)
    go_to_position_parameter(waiting_position)
    move_joint1(17)
    close_tower(floor)

def pick_up_from_tower(floor,position):
    waiting_position = [310.11,5.62,32.55+FLOOR_HEIGHT*floor,-90,0,-90,5.0]
    open_tower(floor)
    move_joint1(-17)
    go_to_position_parameter(waiting_position)
    shift_parallel(x=51,y=-TOWER_DELTA_Y*position)
    shift_parallel(z=6)
    close_hand()
    time.sleep(0.5)
    open_hand()
    shift_parallel(z=-6)
    shift_parallel(x=-51,y=TOWER_DELTA_Y*position)
    go_to_position_parameter(waiting_position)
    move_joint1(17)
    
def set_sample_in_miniflex(floor,position):
    open_hand()
    if get_door_status()=="Close":
        open_door()
    else:
        exit()
    waiting_position = [310.11,5.62,32.55+FLOOR_HEIGHT*floor,-90,0,-90,5.0]
    open_tower(floor)
    move_joint1(-17)
    go_to_position_parameter(waiting_position)
    shift_parallel(x=51,y=-TOWER_DELTA_Y*position)
    shift_parallel(z=6)
    close_hand()
    time.sleep(0.5)
    open_hand()
    shift_parallel(z=-6)
    shift_parallel(x=-51,y=TOWER_DELTA_Y*position)
    go_to_position_parameter(waiting_position)
    move_joint1(17)
    go_to_position_parameter(position_before_miniflex)
    go_to_joint_parameter([18.39087487348178, 0.9160677967274204, 69.41830248122903, 4.853530266116941, 19.73065564704061, -94.57060277605763])
    for i in (0,1,2):
        go_to_position_parameter_at_P(positions_for_miniflex[i])
    change_speed(10,50,100)
    for i in (3,4):
        go_to_position_parameter(positions_for_miniflex[i])
    time.sleep(1)
    go_to_position_parameter(positions_for_miniflex[3])
    change_speed(100,100,100)
    go_to_position_parameter(positions_for_miniflex[2])
    for i in (1,0):
        go_to_position_parameter_at_P(positions_for_miniflex[i])
    go_to_joint_parameter([18.39087487348178, 0.9160677967274204, 69.41830248122903, 4.853530266116941, 19.73065564704061, -94.57060277605763])
    go_to_position_parameter(position_before_miniflex)
    close_tower(floor)
    go_to_position_parameter(COBOTTA_SLEEP_POSITION)
    open_hand()
    close_door()
    print("Sample Set")

def remove_sample_from_miniflex(floor,position):
    open_hand()
    if get_door_status()=="Close":
        open_door()
    else:
        exit()
    open_tower(floor)
    go_to_position_parameter(position_before_miniflex)
    go_to_joint_parameter([18.39087487348178, 0.9160677967274204, 69.41830248122903, 4.853530266116941, 19.73065564704061, -94.57060277605763])
    for i in (0,1,2):
        go_to_position_parameter_at_P(positions_for_miniflex[i])
    change_speed(10,50,100)
    for i in (3,5):
        go_to_position_parameter(positions_for_miniflex[i])
    close_hand()
    time.sleep(0.5)
    open_hand()
    go_to_position_parameter(positions_for_miniflex[3])
    change_speed(100,100,100)
    go_to_position_parameter(positions_for_miniflex[2])
    for i in (1,0):
        go_to_position_parameter_at_P(positions_for_miniflex[i])
    go_to_joint_parameter([18.39087487348178, 0.9160677967274204, 69.41830248122903, 4.853530266116941, 19.73065564704061, -94.57060277605763])
    go_to_position_parameter(position_before_miniflex)
    go_to_position_parameter([294.92,96.03,32.55+FLOOR_HEIGHT*floor,-90,0,-73,5.0])
    place_on_tower(floor,position)
    go_to_position_parameter(COBOTTA_SLEEP_POSITION)
    close_door()
    open_hand()
    print("Sample Removed")

def mount_sample(floor,position,radius_max,rotation_number):
    center = [363.1,2.8-TOWER_DELTA_Y*position,78.05+FLOOR_HEIGHT*floor,-90,0,-90,5.0]
    waiting_position = [310.11,5.62,83.05+FLOOR_HEIGHT*floor,-90,0,-90,5.0]
    open_tower(floor)
    move_joint1(-17)
    go_to_position_parameter(waiting_position)
    go_to_position_parameter(center)
    change_speed(speed=50)
    start_time = time.time()
    while True:
        if time.time() - start_time > 20:
            change_speed(speed=100)
            shift_parallel(z=40)
            break
        move_in_a_spiral(center,radius_max,rotation_number)

In [3]:
# cobotta pad

import pygame
import time
pygame.init() 
j = pygame.joystick.Joystick(0)
j.init()
print("コントローラのボタンを押してください")
parameter_list=[1,5,10]
speed_parameter_list=[10,50,100]
number=2
spped_number=2
clear_error()
change_speed(speed=speed_parameter_list[spped_number],accel=100,decel=100)

try:
    while True:
        events = pygame.event.get()
        for event in events:
            if event.type == pygame.JOYBUTTONDOWN: #ボタンが押された場合
                if j.get_button(0):
                    print("1が押されました")
                    # if spped_number != 0:
                    #     spped_number-=1
                    # change_speed(speed=speed_parameter_list[spped_number],accel=100,decel=100)
                    joint = get_now_joint()
                    next_joint = [joint[0],joint[1],joint[2],joint[3],joint[4],-164.1]
                    go_to_joint_parameter(next_joint)
                    time.sleep(0.1)
                elif j.get_button(1):
                    print("2が押されました")
                    if number!=2:
                        number+=1
                    print("parameter:",parameter_list[number])
                    time.sleep(0.1)
                elif j.get_button(2):
                    print("3が押されました")
                    if number != 0:
                        number-=1
                    print("parameter:",parameter_list[number])
                    time.sleep(0.1)
                elif j.get_button(3):
                    print("4が押されました")
                    joint = get_now_joint()
                    next_joint = [joint[0],joint[1],joint[2],joint[3],joint[4],-90]
                    go_to_joint_parameter(next_joint)
                    time.sleep(0.1)
                elif j.get_button(4):
                    print("L1が押されました")
                    move_joint1(parameter_list[number])
                elif j.get_button(5):
                    print("R1が押されました")
                    move_joint1(-parameter_list[number])
                elif j.get_button(6):
                    print("L2が押されました")
                    close_hand(6)
                    time.sleep(0.1)
                elif j.get_button(7):
                    print("R2が押されました")
                    open_hand()
                    time.sleep(0.1)
                elif j.get_button(8):
                    print("L 押し込み")
                    shift_parallel(0,0,-parameter_list[number]*0.5,0,0,0)
                    time.sleep(0.1)
                elif j.get_button(9):
                    print("R 押し込み")
                    shift_parallel(0,0,parameter_list[number]*0.5,0,0,0)
                    time.sleep(0.1)
                elif j.get_button(10):
                    print("11が押されました")
                    go_to_sleep_position()
                    time.sleep(0.1)
                elif j.get_button(11):
                    print("12が押されました")
                    disconnect()
            elif event.type == pygame.JOYHATMOTION:
                print("十字キー座標")
                print(j.get_hat(0))
                shift_parallel(int(j.get_hat(0)[1]*parameter_list[number]),-int(j.get_hat(0)[0]*parameter_list[number]),0,0,0,0)
except KeyboardInterrupt:
    print("プログラムを終了します")
    j.quit()

ModuleNotFoundError: No module named 'pygame'

In [ ]:
# ボタンチェック

import pygame
import time
pygame.init()
j = pygame.joystick.Joystick(0)
j.init()
print("コントローラのボタンを押してください")
try:
    while True:
        events = pygame.event.get()
        for event in events:
            if event.type == pygame.JOYBUTTONDOWN: #ボタンが押された場合
                if j.get_button(0):
                    print("1が押されました")
                    time.sleep(0.1)
                elif j.get_button(1):
                    print("2が押されました")
                    time.sleep(0.1)
                elif j.get_button(2):
                    print("3が押されました")
                    time.sleep(0.1)
                elif j.get_button(3):
                    print("4が押されました")
                    time.sleep(0.1)
                elif j.get_button(4):
                    print("L1が押されました")
                    time.sleep(0.1)
                elif j.get_button(5):
                    print("L1が押されました")
                    time.sleep(0.1)
                elif j.get_button(6):
                    print("L2が押されました")
                    time.sleep(0.1)
                elif j.get_button(7):
                    print("R2が押されました")
                    time.sleep(0.1)
                elif j.get_button(8):
                    print("L 押し込み")
                    time.sleep(0.1)
                elif j.get_button(9):
                    print("R 押し込み")
                    time.sleep(0.1)
                elif j.get_button(10):
                    print("11が押されました")
                    time.sleep(0.1)
                elif j.get_button(11):
                    print("12が押されました")
                    time.sleep(0.1)
            elif event.type == pygame.JOYHATMOTION:
                print("十字キー座標")
                print(j.get_hat(0))
            elif event.type == pygame.JOYAXISMOTION:
                if abs((j.get_axis(0))) >= 0.5 or abs((j.get_axis(1))) >= 0.5:
                    print("左スティック座標")
                    print(j.get_axis(0),-j.get_axis(1))
                    time.sleep(0.1)
                elif abs((j.get_axis(2))) >= 0.5 or abs((j.get_axis(3))) >= 0.5:
                    print("右スティック座標")
                    # print("("+str(j.get_axis(2))+","+ str(j.get_axis(3))+")")
                    print(j.get_axis(2),j.get_axis(3))
                    time.sleep(0.1)
except KeyboardInterrupt:
    print("プログラムを終了します")
    j.quit()

In [27]:
clear_error()

In [33]:
# move from calset pose
move_joint1(-10)
move_joint2(10)
move_joint3(-10)
move_joint4(10)
move_joint5(-10)
move_joint6(-10)


In [34]:
disconnect()

Disconnection succeeded
